# Домашняя работа по теме "Машинное обучение ранжированию"

В этом ДЗ мы:
- научимся работать со стандартным датасетом для машинного обучения ранжированию [MSLR](https://www.microsoft.com/en-us/research/project/mslr/)
- попробуем применить на практике все то, чему мы научились на семинаре

## Как будет происходить сдача ДЗ

Вам надо:
- форкнуть эту репу
- создать бранч в котором вы дальше будете работать
- реализовать класс Model в этом ноутбуке
- убедиться, что ваша реализация выбивает NDCG@10 выше бейзлайна (см. ниже)
- запушить ваш бранч и поставить Pull Request
- в комментарии написать какой скор вы выбили

В таком случае мы (организаторы):

- счекаутим вашу бранчу
- проверим что ваша реализация действительно выбивает заявленный скор

Предполагается, что и вы, и мы работаем в виртаульном окружении как в семинаре про машинное обучение ранжированию: seminars/7-learning-to-rank/requirements.txt(подробнее про работу с виртуальными окружениями README в корне этой репы).

Оценка:
- За выбитый скор больше **0.507** назначаем **5** баллов, за скор больше (или равно) **0.510** назначаем максимальный балл -- 10 баллов
- Тот из участников кто выбъет самый высокий скор получит еще +10 баллов

При сдаче кода важно помнить о том, что:
- В коде не должно быть захардкоженных с потолка взятых гиперпараметров (таких как число деревьев, learning rate и т.п.) -- обязательно должен быть представлен код который их подбирает!
- Решение должно быть стабильно от запуска к запуску (на CPU) т.е. все seed'ы для генераторов случайных чисел должны быть фиксированы
- Мы (организаторы) будем запускать код на CPU поэтому, даже если вы использовали для подбора параметров GPU, финальный скор надо репортить на CPU

## Пререквизиты

Импортируем все что нам понадобится для дальнейшей работы:

In [133]:
import pathlib
from timeit import default_timer as timer

from functools import partial

import numpy as np
import pandas as pd

import catboost
from catboost import datasets, utils, CatBoostRanker
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
from hyperopt.pyll import scope

## Датасет MSLR (Microsoft Learning to Rank)

Загрузим датасет MSLR.

Полный датасет можно скачать с официального сайта: https://www.microsoft.com/en-us/research/project/mslr/

Строго говоря, он состоит их 2х частей:

- основной датасет MSLR-WEB30K -- он содержит более 30 тыс. запросов
- "маленький" датасет MSLR-WEB10K, который содержит только 10 тыс. запросов и является случайным сэмплом датасета MSLR-WEB30K

в этом ДЗ мы будем работать с MSLR-WEB10K, т.к. полная версия датасета может просто не поместиться у нас в RAM (и, тем более, в память видеокарты если мы учимся на GPU)

Будем считать, что мы самостоятельно скачали датасет MSLR-WEB10K с официального сайта, поместили его в папку КОРЕНЬ-ЭТОЙ-РЕПЫ/data/mslr-web10k и раззиповали.

В результате у нас должна получиться следующая структура папок:

(От студента)Небольшое предупреждение: данный ноутбук выполнялся на Windows, поэтому поехала кодировка в паре клеток ноутбука.

In [66]:
# unix
# !ls ../../data
# win
!dir ..\..\data\

 ’®¬ ў гбва®©бвўҐ C Ё¬ҐҐв ¬ҐвЄг OS
 ‘ҐаЁ©­л© ­®¬Ґа в®¬ : 643C-863E

 ‘®¤Ґа¦Ё¬®Ґ Ї ЇЄЁ C:\Hobby\vk-msu-ir-course-spring-2024\data

19.04.2024  15:49    <DIR>          .
19.04.2024  15:49    <DIR>          ..
19.04.2024  15:33                29 .gitignore
19.04.2024  15:51    <DIR>          mslr-web10k
19.04.2024  15:45     1я234я144я912 MSLR-WEB10K.zip
               2 д ©«®ў  1я234я144я941 Ў ©в
               3 Ї Ї®Є  26я956я156я928 Ў ©в бў®Ў®¤­®


In [67]:
# unix
# !ls -lh ../../data/mslr-web10k/\
# win
!dir ..\..\data\mslr-web10k

 ’®¬ ў гбва®©бвўҐ C Ё¬ҐҐв ¬ҐвЄг OS
 ‘ҐаЁ©­л© ­®¬Ґа в®¬ : 643C-863E

 ‘®¤Ґа¦Ё¬®Ґ Ї ЇЄЁ C:\Hobby\vk-msu-ir-course-spring-2024\data\mslr-web10k

19.04.2024  15:51    <DIR>          .
19.04.2024  15:51    <DIR>          ..
19.04.2024  15:49    <DIR>          Fold1
19.04.2024  15:50    <DIR>          Fold2
19.04.2024  15:50    <DIR>          Fold3
19.04.2024  15:51    <DIR>          Fold4
19.04.2024  15:51    <DIR>          Fold5
               0 д ©«®ў              0 Ў ©в
               7 Ї Ї®Є  26я956я156я928 Ў ©в бў®Ў®¤­®


Заметим, что датасет довольно большой, в распакованном виде он весит 7.7 GB.

Датасет состоит из нескольких фолдов, которые по сути представляют из себя разные разбиения одних и тех же данных на обучающее, валидационное и тестовые множеста.

Дальше мы будем использовать только первый фолд: Fold1.

Заглянем внутрь:

In [68]:
# unix
# !ls -lh ../../data/mslr-web10k/Fold1
# win
!dir ..\..\data\mslr-web10k\Fold1

 ’®¬ ў гбва®©бвўҐ C Ё¬ҐҐв ¬ҐвЄг OS
 ‘ҐаЁ©­л© ­®¬Ґа в®¬ : 643C-863E

 ‘®¤Ґа¦Ё¬®Ґ Ї ЇЄЁ C:\Hobby\vk-msu-ir-course-spring-2024\data\mslr-web10k\Fold1

19.04.2024  15:49    <DIR>          .
19.04.2024  15:49    <DIR>          ..
19.04.2024  15:49       279я254я654 test.txt
19.04.2024  15:49       838я011я150 train.txt
19.04.2024  15:49       272я760я099 vali.txt
               3 д ©«®ў  1я390я025я903 Ў ©в
               2 Ї Ї®Є  26я956я222я464 Ў ©в бў®Ў®¤­®


Видим, что у нас 3 файла с говорящими названиями, соответсвующими сплитам нашего датасета.

Посмотрим на содержимое одного из файлов:

In [69]:
!head -n 1 ../../data/mslr-web10k/Fold1/train.txt

"head" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


Видим, что данные лежат в уже знакомом нам по семинару формате:

- В первой колонке лежит таргет (оценка асессора), по 5-балльной шкале релевантности: от 0 до 4 (включительно)
- Во второй колонке лежит ID запроса, по которому можно сгруппировать все оценки документов в рамках одного и того же запроса
- Дальше идет вектор из 128 фичей (таких как значения BM25 и т.п.), их точная природа нам сейчас на важна

В файле qid и все-фичи кодируются в формате КЛЮЧ:ЗНАЧЕНИЕ, напр. 130:116 -- тут 130 это номер фичи, а 116 -- ее значение.

Такой формат в мире машинного обучения часто называют svm light формат (в честь когда-то популярной библиотеки SVM-Light)

Напишем немного вспомогательного кода для загрузки этого датасета:

In [70]:
def generate_column_names(num_features):
    """Generates column names for LETOR-like datasets"""
    columns = ['label', 'qid']
    for i in range(num_features):
        column = f"feature_{i+1}"
        columns.append(column)
    return columns
    
def load_svmlight_file(input_file, max_num_lines=0):
    """Loads dataset split in SVM-Light format"""
    def _parse_field(field):
        parts = field.split(':')
        if len(parts) != 2:
            raise Exception(f"invalid number of parts in field {field}")
        return parts

    num_features = 136
    exp_num_fields = num_features + 2
    num_lines = 0
    X = []
    with open(input_file, 'rt') as f:
        for line in f:
            try:
                num_lines += 1
                                  
                # Parse into fields
                fields = line.rstrip().split(' ')
                num_fields = len(fields)
                if num_fields != exp_num_fields:
                    raise Exception(f"invalid number of fields {num_fields}")
    
                # Parse every field
                x = np.zeros(exp_num_fields, dtype=np.float32)
                label = int(fields[0])
                x[0] = label
                _, qid_str = _parse_field(fields[1])
                qid = int(qid_str)
                x[1] = qid
                for i, field in enumerate(fields[2:]):
                    _, feature_str = _parse_field(field)
                    x[i+2] = float(feature_str)
    
                # Add new object
                X.append(x)
                if num_lines % 50000 == 0:
                    print(f"Loaded {num_lines} lines...")
                if max_num_lines > 0 and num_lines == max_num_lines:
                    print(f"WARNING: stop loading, line limit reached: max_num_lines = {max_num_lines} input_file = {input_file}")
                    break
            except Exception as e:
                raise Exception(f"error at line {num_lines} in {input_file}") from e
    
    # To pandas
    df = pd.DataFrame(X, columns=generate_column_names(num_features))
    print(f"Loaded SVM-Light file {input_file}")
    return df

И теперь загрузим датасет:

In [71]:
fold_dir = pathlib.Path("../../data/mslr-web10k/Fold1")

df_train = load_svmlight_file(fold_dir.joinpath("train.txt"))
df_valid = load_svmlight_file(fold_dir.joinpath("vali.txt"))
df_test = load_svmlight_file(fold_dir.joinpath("test.txt"))
print(f"Dataset loaded from fold_dir {fold_dir}")

Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded 250000 lines...
Loaded 300000 lines...
Loaded 350000 lines...
Loaded 400000 lines...
Loaded 450000 lines...
Loaded 500000 lines...
Loaded 550000 lines...
Loaded 600000 lines...
Loaded 650000 lines...
Loaded 700000 lines...
Loaded SVM-Light file ..\..\data\mslr-web10k\Fold1\train.txt
Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded SVM-Light file ..\..\data\mslr-web10k\Fold1\vali.txt
Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded SVM-Light file ..\..\data\mslr-web10k\Fold1\test.txt
Dataset loaded from fold_dir ..\..\data\mslr-web10k\Fold1


Посмотрим на данные:

In [72]:
qid_train = set(df_train['qid'].unique().astype('int'))
qid_valid = set(df_valid['qid'].unique().astype('int'))
qid_test = set(df_test['qid'].unique().astype('int'))

In [73]:
print(set.intersection(qid_train, qid_valid))
print(set.intersection(qid_train, qid_test))
print(set.intersection(qid_test, qid_valid))

set()
set()
set()


In [74]:
print(df_valid.head(5))

   label   qid  feature_1  feature_2  feature_3  feature_4  feature_5  \
0    0.0  10.0        2.0        0.0        0.0        0.0        2.0   
1    0.0  10.0        1.0        0.0        1.0        3.0        3.0   
2    1.0  10.0        3.0        0.0        3.0        0.0        3.0   
3    0.0  10.0        3.0        0.0        2.0        0.0        3.0   
4    1.0  10.0        3.0        0.0        3.0        0.0        3.0   

   feature_6  feature_7  feature_8  ...  feature_127  feature_128  \
0   0.666667        0.0   0.000000  ...         45.0          1.0   
1   0.333333        0.0   0.333333  ...         76.0          0.0   
2   1.000000        0.0   1.000000  ...         73.0          0.0   
3   1.000000        0.0   0.666667  ...         54.0          8.0   
4   1.000000        0.0   1.000000  ...         36.0          6.0   

   feature_129  feature_130  feature_131  feature_132  feature_133  \
0          0.0        117.0      55115.0          7.0          2.0   
1     

Т.е. теперь мы видим что данные доступны в точно таком же виде, как это было в семинаре.

Проведем небольшой EDA.

Всего у нас 723 тыс. документов в трейне:

In [75]:
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723412 entries, 0 to 723411
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 380.8 MB
None


235 тыс. документов в валидации:

In [76]:
print(df_valid.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235259 entries, 0 to 235258
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 123.8 MB
None


И 241 тыс. документов в тесте:

In [77]:
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241521 entries, 0 to 241520
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 127.1 MB
None


In [78]:
df_train.sort_values(by='qid', inplace=True)
df_valid.sort_values(by='qid', inplace=True)
df_test.sort_values(by='qid', inplace=True)

In [79]:
df_train['qid'].is_monotonic_increasing, df_valid['qid'].is_monotonic_increasing, df_test['qid'].is_monotonic_increasing

(True, True, True)

Сколько у нас запросов?

In [80]:
num_queries_train = df_train['qid'].nunique()
num_queries_valid = df_valid['qid'].nunique()
num_queries_test = df_test['qid'].nunique()
print(f"Got {num_queries_train} train, {num_queries_valid} valid and {num_queries_test} test queries")

Got 6000 train, 2000 valid and 2000 test queries


## Обучаем модель

Теперь можно приступить непосредственно к обучению модели. 

Объявим класс модели, который надо будем заимлементить в этом ДЗ:

In [164]:
class Model:
    def __init__(self, optim_params=None):
        self.optim_params = optim_params
        self.model = None
        if self.optim_params is not None:
            self.model = catboost.CatBoostRanker(**optim_params)

    def _create_pool(self, df):
        y = df["label"].to_numpy()
        q = df["qid"].astype('int32').to_numpy()
        X = df.drop(["label", "qid"], axis=1).to_numpy()
        pool = catboost.Pool(data=X, label=y, group_id=q)

        return pool
        
    def find_apply_best_params(self, df_train, df_valid):
        def objective(params, pool_train, pool_valid):
            model = CatBoostRanker(**params,
                                   # early_stopping_rounds=100,
                                   random_seed=42)
            model.fit(pool_train, eval_set=pool_valid, use_best_model=True)
            y_true = pool_valid.get_label()
            y_hat = model.predict(pool_valid)
            group_id = pool_valid.get_group_id_hash()
            score = utils.eval_metric(y_true, y_hat, 'NDCG:top=10;type=Exp', group_id=group_id)[0]
            
            return {'loss': 1-score, 'status': STATUS_OK}

        search_space = {'learning_rate': hp.loguniform(label='C', low=-4*np.log(10), high=-1 * np.log(10)), # lr = [0.001, 0.001, ..., 1]
                'bagging_temperature': hp.uniform('bagging_temp', 0.7, 1.1),
                'depth': hp.randint('depth', 4, 10),
                'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1, 10),
                'use_best_model': True,
                'grow_policy': 'Depthwise',
                'loss_function': 'YetiRank',
                'eval_metric': 'NDCG:top=10;type=Exp'
                }
        trials = Trials()

        pool_train = self._create_pool(df_train)
        pool_valid = self._create_pool(df_valid)
        
        best_params = fmin(fn=partial(objective, pool_train=pool_train, pool_valid=pool_valid),
                         space=search_space,
                         algo=tpe.suggest,
                         max_evals=150,
                         trials=trials,
                         rstate=np.random.RandomState(42))

        self.optim_params=trials.results
        self.model = CatBoostRanker(**self.optim_params,
                                   early_stopping_rounds=100,
                                   random_seed=42)
        return self.optim_params

    def fit(self, df_train, df_valid):
        pool_train = self._create_pool(df_train)
        pool_valid = self._create_pool(df_valid)

        # there is no optim params
        if self.model is None:
            self.model = CatBoostRanker(loss_function='YetiRank',
                                        eval_metric='NDCG:top=10;type=Exp',
                                        early_stopping_rounds=10,
                                        iterations=10000,
                                        random_seed=42)
        self.model.fit(pool_train, eval_set=pool_valid, use_best_model=True)

    def predict(self, df_test):
        pool_test = self._create_pool(df_test)
        return self.model.predict(pool_test)

In [ ]:
model = Model()
optim_params = model.find_apply_best_params(df_train, df_valid)

0:	test: 0.2981911	best: 0.2981911 (0)	total: 2s	remaining: 1h 46m 22s                                            

1:	test: 0.3502268	best: 0.3502268 (1)	total: 3.15s	remaining: 1h 23m 49s                                         

2:	test: 0.3575721	best: 0.3575721 (2)	total: 4.23s	remaining: 1h 14m 57s                                         

3:	test: 0.3743866	best: 0.3743866 (3)	total: 5.28s	remaining: 1h 10m 8s                                          

4:	test: 0.3833666	best: 0.3833666 (4)	total: 6.32s	remaining: 1h 7m 5s                                           

5:	test: 0.3940180	best: 0.3940180 (5)	total: 7.36s	remaining: 1h 5m 3s                                           

6:	test: 0.3983034	best: 0.3983034 (6)	total: 8.39s	remaining: 1h 3m 33s                                          

7:	test: 0.4026242	best: 0.4026242 (7)	total: 9.45s	remaining: 1h 2m 36s                                          

8:	test: 0.4047239	best: 0.4047239 (8)	total: 10.5s	remaining: 1h 1m 46s

In [ ]:
filename = "test_catb.pkl"
with open (filename, "wb") as f:
    pickle.dump(model, f)

In [ ]:
print(optim_params)

Создадим и применим модель:

In [109]:
# Fit
start = timer()
model.fit(df_train, df_valid)
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f}")

# Predict
y_hat_test = model.predict(df_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

0:	test: 0.2770984	best: 0.2770984 (0)	total: 991ms	remaining: 48.6s
1:	test: 0.3325224	best: 0.3325224 (1)	total: 1.97s	remaining: 47.3s
2:	test: 0.3338867	best: 0.3338867 (2)	total: 2.92s	remaining: 45.7s
3:	test: 0.3462628	best: 0.3462628 (3)	total: 3.84s	remaining: 44.2s
4:	test: 0.3646393	best: 0.3646393 (4)	total: 4.75s	remaining: 42.7s
5:	test: 0.3695342	best: 0.3695342 (5)	total: 5.64s	remaining: 41.4s
6:	test: 0.3718971	best: 0.3718971 (6)	total: 6.54s	remaining: 40.2s
7:	test: 0.3839662	best: 0.3839662 (7)	total: 7.57s	remaining: 39.8s
8:	test: 0.3850441	best: 0.3850441 (8)	total: 8.59s	remaining: 39.1s
9:	test: 0.3885131	best: 0.3885131 (9)	total: 9.72s	remaining: 38.9s
10:	test: 0.3879574	best: 0.3885131 (9)	total: 10.7s	remaining: 38s
11:	test: 0.3941497	best: 0.3941497 (11)	total: 11.6s	remaining: 36.9s
12:	test: 0.3949795	best: 0.3949795 (12)	total: 12.5s	remaining: 35.7s
13:	test: 0.3979699	best: 0.3979699 (13)	total: 13.5s	remaining: 34.6s
14:	test: 0.4009849	best: 0.4

Теперь, имея предикты, можно посчитать метрики качества:

In [ ]:
def compute_metrics(y_true, y_hat, q):
    # List of metrics to evaluate
    eval_metrics = ['NDCG:top=10;type=Exp']
    
    for eval_metric in eval_metrics:
        scores = utils.eval_metric(y_true, y_hat, eval_metric, group_id=q)
    
        # Print scores
        print(f"metric = {eval_metric} score = {scores[0]:.3f}")

# Get test targets and groups
y_test = df_test['label'].to_numpy()
q_test = df_test['qid'].to_numpy().astype('uint32')
    
# Compute metrics on test
compute_metrics(y_test, y_hat_test, q_test)

Ожидаем, что ваша модель покажет результаты выше бейзлайна!